In [67]:
# imports
from collections import defaultdict, Counter
import re

In [27]:
# utils
def get_input(day: int, test: bool = False):    
    with open(f"input/day{day}{'_test' if test else ''}.txt", "r") as f:
        return f.read()

def get_input_as_rows(day: int, test: bool = False):
    return get_input(day, test).split("\n")


In [28]:
# day 1
## part one
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
ll.sort()
rl.sort()

total_distance = sum([abs(ll[i]- rl[i]) for i in range(len(ll))])
print(total_distance)

## part two
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
c = Counter(rl)
similarity_score = sum([num * c[num] for num in ll])
print(similarity_score)


1830467
26674158


In [62]:
# day2
## part one
input_day2 = get_input_as_rows(2)
rows = [list(map(int, row.split(" "))) for row in input_day2]

safe_counter = 0
for row in rows:
    safe = True
    for i, num in enumerate(row):
        if i > 1 and ((num > row[i-1]) != (row[i-1] > row[i-2])):
            safe = False
            break
        if i != 0 and (abs(num - row[i-1]) > 3 or abs(num - row[i-1]) < 1):
            safe = False
            break
    if safe:
        safe_counter += 1
print(safe_counter)

## part two
input_day2 = get_input_as_rows(2)
rows = [list(map(int, row.split(" "))) for row in input_day2]

def is_safe(arr):
    safe = True
    for i, num in enumerate(arr):
        if i > 1 and ((num > arr[i-1]) != (arr[i-1] > arr[i-2])):
            safe = False
            break
        if i != 0 and (abs(num - arr[i-1]) > 3 or abs(num - arr[i-1]) < 1):
            safe = False
            break
    return safe

safe_counter = 0
for row in rows:
    if is_safe(row):
        safe_counter += 1
    else:
        for i in range(len(row)):
            if is_safe(row[:i] + row[i+1:]):
                safe_counter += 1
                break
    
print(safe_counter)



463
514


In [78]:
# day3
## part one
input_day3 = get_input(3)
pattern = r"mul\((\d{1,3}),(\d{1,3})\)"

matches = re.findall(pattern, input_day3)
result = sum([int(match[0]) * int(match[1]) for match in matches])
print(result)

## part two
patterns = {
    'mul': r"mul\((\d{1,3})\s*,\s*(\d{1,3})\)",
    'do': r"do\(\)",
    'dont': r"don't\(\)"
}

# Find all instructions with their positions
mul_matches = [(m.span(), int(m.group(1)), int(m.group(2))) for m in re.finditer(patterns['mul'], input_day3)]
do_matches = [m.span()[0] for m in re.finditer(patterns['do'], input_day3)]
dont_matches = [m.span()[0] for m in re.finditer(patterns['dont'], input_day3)]

print(do_matches)

# Combine all state changes (do/don't) and sort by position
state_changes = [(pos, True) for pos in do_matches] + [(pos, False) for pos in dont_matches]
state_changes.sort()

# Process multiplications
enabled = True  # Start enabled
result = 0

for (start, end), num1, num2 in mul_matches:
    # Update enabled state based on any do/don't instructions before this mul
    for pos, new_state in state_changes:
        if pos < start:
            enabled = new_state
        else:
            break
            
    if enabled:
        result += num1 * num2

print(result)

185797128
89798695
